## I. Baseline Neural Network Model Performance

Đầu tiên vạch ra những kế hoạch ban đầu (baseline ) cho bài toán này. Chúng ta sẽ bắt đầu bằng việc import những class và function cần thiết.

In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

Ta khởi tạo **random number generator** để bảo đảm rằng lúc nào ta cũng lấy được cùng một kết quả khi khai triển đoạn code này. Điều này cũng giúp khi chúng ta debug.<br>
Mở thử data từ bên ngoài, ta thấy có **60** biến input (X) và **1** biến output (Y).

In [14]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load dataset
url = 'https://raw.githubusercontent.com/duyduc27/ML4_CBD_Robotics_course/master/study/Data%20files/sonar.csv'
dataframe = pd.read_csv(url, header=None)
dataset = dataframe.values
X = dataset[: , :60].astype(float) #input
Y = dataset[: , 60] #output
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


Biến output có loại là string value ( 'M' - mine và 'R' - rock). Chúng ta phải convert sang các giá trị nguyên **0** và **1**. Chúng ta làm được qua class Label Enconder của Sklearn.

In [0]:
# Encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y)

In [16]:
encoded_Y # sau khi label encode

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Giờ chúng ta đã sẵn sàng để tạo mô hình mạng nơ-ron sử dụng Keras. Chúng ta sẽ sử dụng Sklearn để đánh giá mô hình thông qua stratified k-fold cross-validation (StratifiedKFold). Kĩ thuật resampling này sẽ cung cấp sự đo lường cho biểu diễn của mô hình.<br>
Để sử dụng Keras với Sklearn, chúng ta phải sử dụng wrapper KerasClassifier. Class này nhận một hàm và trả lại mô hình mạng nơ-ron của chúng ta. Nó cũng lấy những thông số và mang theo qua việc gọi fit() như số lượng epoch và batch size.<br>
Hãy bắt đầu bằng việc định ra một function mà cái đó tạo ra baseline model của chúng ta. Mô hình của chúng ta sẽ có một hidden layer là fully connected layer với số nơ-ron bằng số input.<br>
Các weights được khởi tạo sử dụng một Gaussian random number nhỏ. Activation function ở đây là Rectifier. Cái output layer chứa chỉ một nơ-ron để tiên đoán. Nó sử dụng activation function là Sigmoid để tạo ra xác suất output trong range từ 0 đến 1. <br>
Cuối cùng, ta sử dụng logarithmic loss function (binary_crossentropy) trong khi train. Loss function này thì phù hợp cho bài toán phân loại nhị phân. Mô hình cũng sử dụng optimizer là Adam cho gradient descent và metrics là accuracy sẽ được thu thập khi mô hình được train.

In [0]:
#base line model
def create_baseline():
  # create model
  model = Sequential()
  model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # compile model
  model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
  return model

Giờ là lúc đánh giá mô hình qua việc sử dụng stratified cross-validation. Chúng ta đưa số lượng training epochs vào KerasClassifier. Phần Verbose output  bị tắt. (nếu mở sẽ hiện ra quá trình 10 lần cho 10-fold của cross validation)

In [18]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0 )
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print('Baseline %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))

W0704 02:04:17.807852 140218976675712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0704 02:04:17.853160 140218976675712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0704 02:04:17.861752 140218976675712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0704 02:04:17.890213 140218976675712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0704 02:04:17.912739 140218976675712 deprecation_wrapper.

Baseline 80.28% (7.52%)


Mô hình có độ chính xác trung bình  là 80.28% và giao động ở mức 7.52%

## II.Improve Performance With Data Preparation

Sẽ là rất thiết thực nếu bạn chuẩn bị lại dữ liệu trước khi lên mô hình. Mô hình mạng nơ-ron thì _đặc biệt phù hợp_ với khi có những giá trị **input nhất quán**. Việc cách hiệu quả cho việc phối hợp một bộ dữ liệu dạng bảng khi xây dựng một mô hình mạng nơ-ron là chuẩn hóa dữ liệu (standardization). Ở đây dữ liệu sẽ được tỷ lệ lại với giá trị mean trên từng attribute bằng **0** và std bằng **1**. Điều này hợp với các phân phối Gaussian và giống Gaussian trong việc chuẩn hóa lại trọng tâm khuynh hướng của từng attribute.

In [20]:
# Binary Classification with Sonar Dataset: Standardized
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
url = 'https://raw.githubusercontent.com/duyduc27/ML4_CBD_Robotics_course/master/study/Data%20files/sonar.csv'
dataframe = pd.read_csv(url, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:, :60].astype(float) #input
Y = dataset[:, 60] #output
# encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y)
# baseline model
def create_baseline():
  #create model
  model = Sequential()
  model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # compile model
  model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
  return model

# evaluate baseline model with standardized dataset <-- KHÁC BIỆT Ở ĐÂY
estimator = []
estimator.append(('standardize', StandardScaler())) # <-- The difference HERE
estimator.append(('MultiLayer Perceptrons', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimator)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print('Standardized: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))

Standardized: 84.11% (4.36%)


Chúng ta chỉ thay đổi rất nhỏ là chuẩn hóa lại dữ liệu, nhưng kết quả nâng lên rất tốt từ 80 lên 84% !

## III. Tuning Layers and Neurons in The Model
Có nhiều thứ để tune trong một mạng nơ-ron như là weight khởi tạo, activation functions, optimizer, vâng vâng. Ở trong phần này ta sẽ đi qua hai trải nghiệm trên cấu trúc của mạng này là: làm nó _nhỏ đi_ và làm nó  _lớn hơn_.

### 3.1 Evaluate a Smaller Network

Câu hỏi đặt ra ở đây: Tôi nghi ngờ rằng có nhiều biến input dư thừa cho bài toán này. Mô tả dữ liệu cho thấy nó có cùng tín hiệu ở các góc khác nhau. Có lẽ, có nhiều góc thì liên quan hơn những cái còn lại. Chúng ta có thể tập trung vào những loại feature quan trọng này trong mạng bằng việc **giới hạn** khoảng không của hidden layer đầu tiên.<br>
Ở trải nghiệm này chúng ta sẽ sử dụng lại baseline model nhưng chỉ với 30 nơ-ron ở hidden layer đầu tiên thay vì 60 như ban đầu. Điều này sẽ đẩy áp lực lên mạng trong quá trình train để chọn ra những cấu trúc quan trọng nhất ở đầu vào (input) của mô hình. Và chúng ta cũng sẽ chuẩn hóa dữ liệu như ở ví dụ trước để có thêm lợi thế.<br>
Ví dụ bên dưới đơn thuần chỉ thay đổi số nơ-ron ở lớp hidden layer đầu tiên, các biến được giữ nguyên và sử dụng lại ở ví dụ trên, để xem sự thay đổi.

In [21]:
# Binary Classification with Sonar Dataset: Standardized Smaller

# smaller model
def create_smaller():
  # create model
  model = Sequential()
  model.add(Dense(30, input_dim=60, kernel_initializer='normal', activation='relu'))  # <- reduce neurons into 30
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size =5, verbose=0)))
pipeline = Pipeline(estimators)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print('Smaller: %.2f%% (%.2f%%)' % ( results.mean()*100, results.std()*100))

Smaller: 85.52% (5.85%)


Vậy là sau khi giảm số nơ-ron , độ chính xác của mô hình đã tăng thêm được một chút!

### 3.2 Evaluate a Larger Network

Một mạng nơ-ron với nhiều layer hơn thường có nhiều cơ hội để mạng này nhả ra được những feature trọng yếu và nối lại chúng theo các cách phi tuyến tính hữu ích. Chúng ta có thể thêm nhiều layer hơn vào mạng để xem thử sự cải thiện.<br>
Ở ví dụ này, ta thêm 1 hidden layer mới sau hidden layer đầu tiên, và layer mới này có 30 nơ-ron.<br>
Ý tưởng ở đây là cho mạng của chúng ta thêm cơ hội mô hình tất cả các biến trước khi bị 'nén cổ chai' và bị ép vào khuôn khổ giới hạn, giống như chúng ta đã làm với ví dụ ở trên - là làm nhỏ mạng đi.

In [22]:
# larger model
def create_larger():
  #create model
  model = Sequential()
  model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

estimators = []
estimators.append(('Standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print('Larger: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))

Larger: 84.57% (5.24%)


Trong ví dụ này chúng ta có thể thấy rằng độ biểu diễn của mô hình không tăng lên thêm. Điều này có thể do số nhiễu do việc train này. <br>
Xa hơn nữa là việc điều chỉnh optimizer, số lần train epochs . Chúng được mong đợi là có thể cải thiện thêm cho mô hình.